In [2]:
import numpy as np
from itertools import starmap
import pandas as pd 
import xarray as xr
from scipy.io import loadmat, savemat
%pwd

'C:\\Users\\Admin\\github_upload'

In [ ]:
mode_parameter = loadmat(r"D:\metrics_io.mat")
xg, yg, rf = mode_parameter['xg'], mode_parameter['yg'], mode_parameter['rf'].squeeze()
lon_all = xr.DataArray(xg[:, 1], coords=dict(lon=xg[:, 1]))
lat_all = xr.DataArray(yg[-1, :], coords=dict(lat=yg[-1, :]))
depth_all = xr.DataArray(rf, coords=dict(depth=-rf))

time = pd.date_range('2001-01', '2020-12', freq='MS')
year = time.year
month = time.month

def load_surface_SUVTdata(which, year, month):
    filename = r'D:\monthly_mean\{}_mm_{}_{:02d}.mat'.format(which, year, month)
    data = xr.DataArray(loadmat(filename)[which+'0'],
                        coords=dict(lon=lon_all, lat=lat_all, depth=depth_all))
    data = data.sel(lon=slice(40, 110), lat=slice(-15, 31), depth=0).drop_vars('depth')
    time_point = pd.to_datetime(str(year)+'-'+'{}'.format(month))
    data = data.expand_dims(time=[time_point])
    lon, lat = data.lon.sel(lon=slice(40, 110)), data.lat.sel(lat=slice(-15, 31))
    return data, lon, lat

_, lon, lat = load_surface_SUVTdata('t', 2002, 1)


for year in range(2001, 2021):
    print(year)
    for month in range(1, 13):
        params = [
            ('s', year, month), 
            ('u', year, month),
            ('v', year, month),
            ('t', year, month)]
        SUVT_data = list(starmap(load_surface_SUVTdata, params)) 
        #starmap和map函数类似，可以将输入函数映射到多组输入变量。
        SUVT_data[0][0].to_netcdf(
            r'D:\all_processed_data\surface_ncdata_monthly\{}_mm_S_{:02d}.nc'.format(year, month))
        SUVT_data[1][0].to_netcdf(
            r'D:\all_processed_data\surface_ncdata_monthly\{}_mm_U_{:02d}.nc'.format(year, month))
        SUVT_data[2][0].to_netcdf(
            r'D:\all_processed_data\surface_ncdata_monthly\{}_mm_V_{:02d}.nc'.format(year, month))   
        SUVT_data[3][0].to_netcdf(
            r'D:\all_processed_data\surface_ncdata_monthly\{}_mm_T_{:02d}.nc'.format(year, month))               
 